In [1]:
import pandas as pd

train_df = pd.read_csv('data/train.csv')
val_df = pd.read_csv('data/val.csv')
test_df = pd.read_csv('data/test.csv')

In [2]:
df1 = train_df.append(val_df, ignore_index=True).append(test_df, ignore_index=True)

len(df1)

14478

In [3]:
df1.rename(columns={'camera': 'source'}, inplace=True)
df1.head()

,image_path,label,subject,source
0,data/Camera 1/train/c0/1000.jpg,0,7,1
1,data/Camera 1/train/c0/1001.jpg,0,7,1
2,data/Camera 1/train/c0/1002.jpg,0,7,1
3,data/Camera 1/train/c0/1003.jpg,0,7,1
4,data/Camera 1/train/c0/1004.jpg,0,7,1


In [4]:
df1['subject'] = df1.subject.astype(str)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14478 entries, 0 to 14477
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_path  14478 non-null  object
 1   label       14478 non-null  int64 
 2   subject     14478 non-null  object
 3   source      14478 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 452.6+ KB


In [5]:
df1.loc[df1.source == 1, 'source'] = 'Camera 1'
df1.loc[df1.source == 2, 'source'] = 'Camera 2'
df1['subject'] = 'p' + df1.subject
df1.head()

,image_path,label,subject,source
0,data/Camera 1/train/c0/1000.jpg,0,p7,Camera 1
1,data/Camera 1/train/c0/1001.jpg,0,p7,Camera 1
2,data/Camera 1/train/c0/1002.jpg,0,p7,Camera 1
3,data/Camera 1/train/c0/1003.jpg,0,p7,Camera 1
4,data/Camera 1/train/c0/1004.jpg,0,p7,Camera 1


In [6]:
import glob

df2 = []
for filepath in glob.glob('data/StateFarm/*/*'):
    classId = filepath.split('/')[-2][1:]
    df2.append({
        'image_path': filepath,
        'label': int(classId)
    })
    
df2 = pd.DataFrame(df2)
df2.head()

,image_path,label
0,data/StateFarm/c0/img_100026.jpg,0
1,data/StateFarm/c0/img_10003.jpg,0
2,data/StateFarm/c0/img_100050.jpg,0
3,data/StateFarm/c0/img_100074.jpg,0
4,data/StateFarm/c0/img_10012.jpg,0


In [8]:
len(df2) + len(df1)

36902

In [10]:
subjects = pd.read_csv('driver_imgs_list.csv')

In [11]:
subjects.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [12]:
subjects['img'] = 'data/StateFarm/' + subjects.classname + '/' + subjects.img

subjects.head()

,subject,classname,img
0,p002,c0,data/StateFarm/c0/img_44733.jpg
1,p002,c0,data/StateFarm/c0/img_72999.jpg
2,p002,c0,data/StateFarm/c0/img_25094.jpg
3,p002,c0,data/StateFarm/c0/img_69092.jpg
4,p002,c0,data/StateFarm/c0/img_92629.jpg


In [13]:
df2 = pd.merge(df2, subjects, left_on='image_path', right_on='img', sort=False)

df2.head()

,image_path,label,subject,classname,img
0,data/StateFarm/c0/img_100026.jpg,0,p015,c0,data/StateFarm/c0/img_100026.jpg
1,data/StateFarm/c0/img_10003.jpg,0,p022,c0,data/StateFarm/c0/img_10003.jpg
2,data/StateFarm/c0/img_100050.jpg,0,p022,c0,data/StateFarm/c0/img_100050.jpg
3,data/StateFarm/c0/img_100074.jpg,0,p051,c0,data/StateFarm/c0/img_100074.jpg
4,data/StateFarm/c0/img_10012.jpg,0,p002,c0,data/StateFarm/c0/img_10012.jpg


In [14]:
df2 = df2[['image_path', 'label', 'subject']]
df2['source'] = 'StateFarm'

df2.head()

,image_path,label,subject,source
0,data/StateFarm/c0/img_100026.jpg,0,p015,StateFarm
1,data/StateFarm/c0/img_10003.jpg,0,p022,StateFarm
2,data/StateFarm/c0/img_100050.jpg,0,p022,StateFarm
3,data/StateFarm/c0/img_100074.jpg,0,p051,StateFarm
4,data/StateFarm/c0/img_10012.jpg,0,p002,StateFarm


In [15]:
df = df1.append(df2)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36902 entries, 0 to 22423
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_path  36902 non-null  object
 1   label       36902 non-null  int64 
 2   subject     36902 non-null  object
 3   source      36902 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [17]:
df.subject.nunique()

71

In [18]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, train_size=.6, random_state=42)

for train_idx, val_test_idx in gss.split(df, groups=df.subject):
    val_test_df = df.iloc[val_test_idx]
    train_df = df.iloc[train_idx]

In [19]:
gss = GroupShuffleSplit(n_splits=1, train_size=.5, random_state=42)

for val_idx, test_idx in gss.split(val_test_df, groups=val_test_df.subject):
    test_df = val_test_df.iloc[test_idx]
    val_df = val_test_df.iloc[val_idx]

In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()


def explore(df):
    plt.figure(figsize=(15, 10))
    # class distribution
    sns.countplot(x='label', data=df)
    plt.title(f'Class Distribution')
    plt.show()

In [21]:
explore(train_df)

In [22]:
explore(val_df)

In [23]:
explore(test_df)

In [26]:
train_df.subject.nunique(), len(train_df)

(42, 21949)

In [28]:
val_df.subject.nunique(), len(val_df)

(14, 7215)

In [29]:
test_df.subject.nunique(), len(test_df)

(15, 7738)

In [31]:
from collections import Counter
print(Counter(train_df.source))
print(Counter(val_df.source))
print(Counter(test_df.source))

Counter({'StateFarm': 13504, 'Camera 1': 6645, 'Camera 2': 1800})
Counter({'StateFarm': 4495, 'Camera 1': 1920, 'Camera 2': 800})
Counter({'StateFarm': 4425, 'Camera 1': 3113, 'Camera 2': 200})


In [33]:
print(Counter(train_df.subject))
print(Counter(val_df.subject))
print(Counter(test_df.subject))

Counter({'p022': 1233, 'p026': 1196, 'p049': 1011, 'p051': 920, 'p014': 876, 'p015': 875, 'p047': 835, 'p012': 823, 'p081': 823, 'p064': 820, 'p075': 814, 'p061': 809, 'p056': 794, 'p045': 724, 'p041': 605, 'p8': 550, 'p10': 515, 'p5': 461, 'p15': 445, 'p19': 429, 'p7': 420, 'p6': 398, 'p20': 392, 'p2': 360, 'p9': 358, 'p12': 357, 'p1': 354, 'p22': 352, 'p072': 346, 'p21': 340, 'p23': 326, 'p3': 301, 'p25': 287, 'p32': 200, 'p42': 200, 'p44': 200, 'p37': 200, 'p40': 200, 'p43': 200, 'p33': 200, 'p45': 200, 'p38': 200})
Counter({'p021': 1237, 'p024': 1226, 'p050': 790, 'p039': 651, 'p042': 591, 'p28': 531, 'p29': 439, 'p17': 346, 'p27': 335, 'p11': 269, 'p34': 200, 'p35': 200, 'p36': 200, 'p41': 200})
Counter({'p016': 1078, 'p066': 1034, 'p035': 848, 'p052': 740, 'p002': 725, 'p26': 431, 'p14': 411, 'p13': 400, 'p24': 381, 'p18': 367, 'p30': 320, 'p31': 281, 'p4': 280, 'p16': 242, 'p39': 200})


In [35]:
df_images = list(train_df.image_path) + list(val_df.image_path) + list(test_df.image_path)
df_images.sort()
len(df_images)

36902

In [36]:
all_images = glob.glob('data/*/*/*/*.jpg') + glob.glob('data/*/*/*.jpg')
all_images.sort()
len(all_images)

36902

In [40]:
from tqdm import tqdm

for img1, img2 in tqdm(zip(all_images, df_images)):
    print(img1, img2)
    if img1 != img2:
        print('Error!')

71it [00:00, 403.85it/s]

data/Camera 1/test/c0/0.jpg data/Camera 1/test/c0/0.jpg
data/Camera 1/test/c0/1.jpg data/Camera 1/test/c0/1.jpg
data/Camera 1/test/c0/10.jpg data/Camera 1/test/c0/10.jpg
data/Camera 1/test/c0/100.jpg data/Camera 1/test/c0/100.jpg
data/Camera 1/test/c0/101.jpg data/Camera 1/test/c0/101.jpg
data/Camera 1/test/c0/102.jpg data/Camera 1/test/c0/102.jpg
data/Camera 1/test/c0/103.jpg data/Camera 1/test/c0/103.jpg
data/Camera 1/test/c0/104.jpg data/Camera 1/test/c0/104.jpg
data/Camera 1/test/c0/105.jpg data/Camera 1/test/c0/105.jpg
data/Camera 1/test/c0/106.jpg data/Camera 1/test/c0/106.jpg
data/Camera 1/test/c0/107.jpg data/Camera 1/test/c0/107.jpg
data/Camera 1/test/c0/108.jpg data/Camera 1/test/c0/108.jpg
data/Camera 1/test/c0/109.jpg data/Camera 1/test/c0/109.jpg
data/Camera 1/test/c0/11.jpg data/Camera 1/test/c0/11.jpg
data/Camera 1/test/c0/110.jpg data/Camera 1/test/c0/110.jpg
data/Camera 1/test/c0/111.jpg data/Camera 1/test/c0/111.jpg
data/Camera 1/test/c0/112.jpg data/Camera 1/test/c0/

1247it [00:00, 3902.14it/s]

data/Camera 1/test/c2/121111.jpg data/Camera 1/test/c2/121111.jpg
data/Camera 1/test/c2/122111.jpg data/Camera 1/test/c2/122111.jpg
data/Camera 1/test/c2/123111.jpg data/Camera 1/test/c2/123111.jpg
data/Camera 1/test/c2/124111.jpg data/Camera 1/test/c2/124111.jpg
data/Camera 1/test/c2/125111.jpg data/Camera 1/test/c2/125111.jpg
data/Camera 1/test/c2/126111.jpg data/Camera 1/test/c2/126111.jpg
data/Camera 1/test/c2/127111.jpg data/Camera 1/test/c2/127111.jpg
data/Camera 1/test/c2/128111.jpg data/Camera 1/test/c2/128111.jpg
data/Camera 1/test/c2/129111.jpg data/Camera 1/test/c2/129111.jpg
data/Camera 1/test/c2/130111.jpg data/Camera 1/test/c2/130111.jpg
data/Camera 1/test/c2/13111.jpg data/Camera 1/test/c2/13111.jpg
data/Camera 1/test/c2/131111.jpg data/Camera 1/test/c2/131111.jpg
data/Camera 1/test/c2/132111.jpg data/Camera 1/test/c2/132111.jpg
data/Camera 1/test/c2/133111.jpg data/Camera 1/test/c2/133111.jpg
data/Camera 1/test/c2/14111.jpg data/Camera 1/test/c2/14111.jpg
data/Camera 1/

2319it [00:00, 4603.40it/s]

...

data/StateFarm/c8/img_90126.jpg data/StateFarm/c8/img_90126.jpg
data/StateFarm/c8/img_90151.jpg data/StateFarm/c8/img_90151.jpg
data/StateFarm/c8/img_90239.jpg data/StateFarm/c8/img_90239.jpg
data/StateFarm/c8/img_90242.jpg data/StateFarm/c8/img_90242.jpg
data/StateFarm/c8/img_90349.jpg data/StateFarm/c8/img_90349.jpg
data/StateFarm/c8/img_90486.jpg data/StateFarm/c8/img_90486.jpg
data/StateFarm/c8/img_90501.jpg data/StateFarm/c8/img_90501.jpg
data/StateFarm/c8/img_90536.jpg data/StateFarm/c8/img_90536.jpg
data/StateFarm/c8/img_90540.jpg data/StateFarm/c8/img_90540.jpg
data/StateFarm/c8/img_90567.jpg data/StateFarm/c8/img_90567.jpg
data/StateFarm/c8/img_90621.jpg data/StateFarm/c8/img_90621.jpg
data/StateFarm/c8/img_90667.jpg data/StateFarm/c8/img_90667.jpg
data/StateFarm/c8/img_90703.jpg data/StateFarm/c8/img_90703.jpg
data/StateFarm/c8/img_90746.jpg data/StateFarm/c8/img_90746.jpg
data/StateFarm/c8/img_90786.jpg data/StateFarm/c8/img_90786.jpg
data/StateFarm/c8/img_90873.jpg data/Sta

36463it [00:07, 5298.52it/s]

data/StateFarm/c9/img_46776.jpg
data/StateFarm/c9/img_46805.jpg data/StateFarm/c9/img_46805.jpg
data/StateFarm/c9/img_46872.jpg data/StateFarm/c9/img_46872.jpg
data/StateFarm/c9/img_46908.jpg data/StateFarm/c9/img_46908.jpg
data/StateFarm/c9/img_46911.jpg data/StateFarm/c9/img_46911.jpg
data/StateFarm/c9/img_46995.jpg data/StateFarm/c9/img_46995.jpg
data/StateFarm/c9/img_47018.jpg data/StateFarm/c9/img_47018.jpg
data/StateFarm/c9/img_47026.jpg data/StateFarm/c9/img_47026.jpg
data/StateFarm/c9/img_47062.jpg data/StateFarm/c9/img_47062.jpg
data/StateFarm/c9/img_47067.jpg data/StateFarm/c9/img_47067.jpg
data/StateFarm/c9/img_47069.jpg data/StateFarm/c9/img_47069.jpg
data/StateFarm/c9/img_47074.jpg data/StateFarm/c9/img_47074.jpg
data/StateFarm/c9/img_47080.jpg data/StateFarm/c9/img_47080.jpg
data/StateFarm/c9/img_47109.jpg data/StateFarm/c9/img_47109.jpg
data/StateFarm/c9/img_47121.jpg data/StateFarm/c9/img_47121.jpg
data/StateFarm/c9/img_47149.jpg data/StateFarm/c9/img_47149.jpg
data/Sta

36902it [00:07, 5178.97it/s]

data/StateFarm/c9/img_9378.jpg
data/StateFarm/c9/img_93787.jpg data/StateFarm/c9/img_93787.jpg
data/StateFarm/c9/img_93807.jpg data/StateFarm/c9/img_93807.jpg
data/StateFarm/c9/img_93821.jpg data/StateFarm/c9/img_93821.jpg
data/StateFarm/c9/img_93880.jpg data/StateFarm/c9/img_93880.jpg
data/StateFarm/c9/img_93918.jpg data/StateFarm/c9/img_93918.jpg
data/StateFarm/c9/img_93973.jpg data/StateFarm/c9/img_93973.jpg
data/StateFarm/c9/img_93995.jpg data/StateFarm/c9/img_93995.jpg
data/StateFarm/c9/img_94109.jpg data/StateFarm/c9/img_94109.jpg
data/StateFarm/c9/img_94148.jpg data/StateFarm/c9/img_94148.jpg
data/StateFarm/c9/img_94341.jpg data/StateFarm/c9/img_94341.jpg
data/StateFarm/c9/img_94352.jpg data/StateFarm/c9/img_94352.jpg
data/StateFarm/c9/img_94377.jpg data/StateFarm/c9/img_94377.jpg
data/StateFarm/c9/img_94439.jpg data/StateFarm/c9/img_94439.jpg
data/StateFarm/c9/img_9460.jpg data/StateFarm/c9/img_9460.jpg
data/StateFarm/c9/img_94696.jpg data/StateFarm/c9/img_94696.jpg
data/StateF

In [41]:
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)
test_df.to_csv('test.csv', index=False)